# Chat Prompts Customization

## Prompt Setup

Below, we take the default prompts and customize them to always answer, even if the context is not helpful.

In [2]:
from langchain.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from llama_index.prompts import Prompt

chat_text_qa_msgs = [
    SystemMessagePromptTemplate.from_template("Always answer the question, even if the context isn't helpful."),
    HumanMessagePromptTemplate.from_template(
        "Context information is below.\n"
        "---------------------\n"
        "{context_str}\n"
        "---------------------\n"
        "Given the context information and not prior knowledge, "
        "answer the question: {query_str}\n"
    )
]
chat_text_qa_msgs_lc = ChatPromptTemplate.from_messages(chat_text_qa_msgs)
text_qa_template = Prompt.from_langchain_prompt(chat_text_qa_msgs_lc)

# Refine Prompt
chat_refine_msgs = [
    SystemMessagePromptTemplate.from_template("Always answer the question, even if the context isn't helpful."),
    HumanMessagePromptTemplate.from_template(
        "We have the opportunity to refine the original answer "
        "(only if needed) with some more context below.\n"
        "------------\n"
        "{context_msg}\n"
        "------------\n"
        "Given the new context, refine the original answer to better "
        "answer the question: {query_str}. "
        "If the context isn't useful, output the original answer again.\n"
        "Original Answer: {existing_answer}"
    ),
]


chat_refine_msgs_lc = ChatPromptTemplate.from_messages(chat_refine_msgs)
refine_template = Prompt.from_langchain_prompt(chat_refine_msgs_lc)

## Using the Prompts

Now, we use the prompts in an index query!

In [3]:
import openai
import os

os.environ["OPENAI_API_KEY"] = "YOUR_API_KEY"
openai.api_key = os.environ["OPENAI_API_KEY"]

In [4]:
from llama_index import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.llms import OpenAI

documents = SimpleDirectoryReader("../../data/paul_graham/").load_data()

# Create an index using a chat model, so that we can use the chat prompts!
service_context = ServiceContext.from_defaults(llm=OpenAI(model="gpt-3.5-turbo", temperature=0.0))

index = VectorStoreIndex.from_documents(documents, service_context=service_context)

### Before Adding Templates

In [5]:
print(index.as_query_engine().query("Who is Joe Biden?"))

Based on the given context information, there is no mention of Joe Biden. Therefore, it is not possible to determine who Joe Biden is based on this information alone.


### After Adding Templates

In [6]:
print(index.as_query_engine(text_qa_template=text_qa_template, refine_template= refine_template).query("Who is Joe Biden?"))

Joe Biden is a politician who served as the 46th President of the United States.
